In [14]:
import numpy as np
import os
import string
import random
import csv

# Function to convert a key to its binary hash representation
def hash_function(key):
    return '{0:016b}'.format(key)

#Initializes bucket
class Bucket:
    def __init__(self, local_depth, index, empty_spaces, id):
        self.id = id #Bucket ID
        self.local_depth = local_depth #Local depth of bucket
        self.index = index #List of numbers in a bucket
        self.empty_spaces = empty_spaces #Empty spaces in a bucket

#Initializes directory
class Directory:
    def __init__(self, global_depth, directory_records):
        self.global_depth = global_depth #Global depth of directory
        self.directory_records = directory_records #List of directory records

#Initializes directory record        
class DirectoryRecord:
    def __init__(self, bucket, hash_prefix):
        self.hash_prefix = hash_prefix #Hash prefix for directory record
        self.value = bucket #Bucket associated with directory record

bucket_capacity = 3 #Capacity of each bucket
bucket_number = 2 #Starting bucket ID's
global_depth = 1 #Initial global depth

# Initialization of buckets
bucket1 = Bucket(local_depth=1, empty_spaces=bucket_capacity, index=[], id=1)
bucket2 = Bucket(local_depth=1, empty_spaces=bucket_capacity, index=[], id=2)

# Initialization of directory records
directory_records = list()
directory_records.append(DirectoryRecord(hash_prefix=0, bucket=bucket1))
directory_records.append(DirectoryRecord(hash_prefix=1, bucket=bucket2))

#Initialize directory
directory = Directory(global_depth=1, directory_records=directory_records)


def insert(index):
    
    global directory
    global bucket_number
    
    t_id = index[0] # Gets ID from index
    hash_key = hash_function(int(t_id)) # Converts ID into binary
    hash_prefix = int(hash_key[-directory.global_depth:], 2) # Compute hash prefix based on global depth

    bucket = directory.directory_records[hash_prefix].value # Get corresponding bucket
    bucket.index.append(index) # Add index to the bucket
    bucket.empty_spaces -= 1 # Decrease the empty spaces in the bucket

    # If bucket is overloaded handle overflow
    if bucket.empty_spaces < 0: 
        temp_memory = bucket.index[:]
        bucket.empty_spaces = bucket_capacity
        bucket.index = []
        
        if directory.global_depth > bucket.local_depth:
            number_of_links = 2 ** (directory.global_depth - bucket.local_depth)
            bucket.local_depth += 1
            number_of_modify_links = number_of_links // 2

            new_bucket = Bucket(local_depth=bucket.local_depth, index=[], empty_spaces=bucket_capacity, id=bucket_number)
            bucket_number += 1

            for directory_record in directory.directory_records:
                if directory_record.value == bucket:
                    if number_of_modify_links != 0:
                        number_of_modify_links -= 1
                    else:
                        directory_record.value = new_bucket

            for item in temp_memory:
                insert(item)

        elif directory.global_depth == bucket.local_depth:
            new_directory_len = 2 * len(directory.directory_records)
            new_directory_records = []

            for directory_record_number in range(new_directory_len):
                new_directory_records.append(DirectoryRecord(
                    hash_prefix=directory_record_number, 
                    bucket=Bucket(local_depth=1, index=[], empty_spaces=bucket_capacity, id=bucket_number)
                ))
                bucket_number += 1
            
            new_directory = Directory(global_depth=directory.global_depth + 1, directory_records=new_directory_records)

            for directory_record in directory.directory_records:
                hashkey1 = '0' + hash_function(directory_record.hash_prefix)
                hashkey2 = '1' + hash_function(directory_record.hash_prefix)
                new_index1 = int(hashkey1[-new_directory.global_depth:], 2)
                new_index2 = int(hashkey2[-new_directory.global_depth:], 2)

                new_directory.directory_records[new_index1].value = directory_record.value
                new_directory.directory_records[new_index2].value = directory_record.value

            directory = new_directory

            for item in temp_memory:
                insert(item)
    print_directory()
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        for record in directory.directory_records:
            temp_memory.extend(record.value.index)
            record.value.index = [:] # Clear the bucket
            record.value.empty_spaces = bucket_capacity # Reset empty spaces

        # Store the overflowing index
        temp_memory.append(index)
        
        if directory.global_depth > bucket.local_depth: 
            number_of_links = 2 ** (directory.global_depth - bucket.local_depth) # Calculate how many directory records point to the bucket
            bucket.local_depth += 1 # Increase the local depth of the bucket
            number_of_modify_links = number_of_links // 2 # Calculates the new number of links for the bucket
            
            # Creates new bucket
            new_bucket = Bucket(local_depth=bucket.local_depth, index=[], empty_spaces=bucket_capacity, id=bucket_number)
            bucket_number += 1
            
            # Updates directory records with new bucket references
            for directory_record in directory.directory_records: # Iterates over each directory record
                if directory_record.value == bucket: # Check if each directory record references the bucket that overflowed
                    if number_of_modify_links != 0: # If there are still links to modify
                        number_of_modify_links -= 1 
                    else:
                        directory_record.value = new_bucket # Assign new bucket to directory record
            
        elif directory.global_depth == bucket.local_depth: 
            directory.global_depth += 1
            new_directory_records = []
            for record in directory.directory_records:
                new_directory_records.append(DirectoryRecord(hash_prefix=record.hash_prefix, bucket=record.value))
                new_directory_records.append(DirectoryRecord(hash_prefix=record.hash_prefix, bucket=record.value))
            
            directory.directory_records = new_directory_records
            
        # Reinsert all items
        for item in temp_memory:
            insert(item)
    else:
        print_directory()



    
def print_directory():
    global directory
    print("Directory State:")
    for record in directory.directory_records:
        print(f"Binary: {record.hash_prefix}, Bucket ID: {record.value.id}, Local Depth: {record.value.local_depth}, Values: {record.value.index}")
    print()    
    

# Inserting values and printing the directory state
#for value in values:
insert([16])

SyntaxError: invalid syntax (3922091208.py, line 70)

In [2]:
insert([22])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [22]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: []



In [3]:
insert([26])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [22], [26]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: []



In [4]:
insert([20])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16]]
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: []
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: []

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16]]
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[22]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[22]]

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16]]
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[22], [26]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[22], [26]]

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20]]
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[22], [26]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[22], [26]]

Dire

In [9]:
insert([3])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [20]]
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [20]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[22], [26], [3]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[22], [26], [3]]



In [25]:
insert([1])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26]]
Binary: 3, Bucket ID: 5, Local Depth: 1, Values: [[3]]



In [26]:
insert([12])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26]]
Binary: 3, Bucket ID: 5, Local Depth: 1, Values: [[3]]



In [27]:
insert([11])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26]]
Binary: 3, Bucket ID: 5, Local Depth: 1, Values: [[3], [11]]



In [28]:
insert([13])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1], [13]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26]]
Binary: 3, Bucket ID: 5, Local Depth: 1, Values: [[3], [11]]



In [29]:
insert([19])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1], [13]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26]]
Binary: 3, Bucket ID: 5, Local Depth: 1, Values: [[3], [11], [19]]



In [30]:
insert([38])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1], [13]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26], [38]]
Binary: 3, Bucket ID: 5, Local Depth: 1, Values: [[3], [11], [19]]



In [31]:
insert([47])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1], [13]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26], [38]]
Binary: 3, Bucket ID: 5, Local Depth: 2, Values: [[3]]

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1], [13]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26], [38]]
Binary: 3, Bucket ID: 5, Local Depth: 2, Values: [[3], [11]]

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1], [13]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26], [38]]
Binary: 3, Bucket ID: 5, Local Depth: 2, Values: [[3], [11], [19]]

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1], [13]]
Binary: 2, Bucket ID: 4, L

In [32]:
insert([46])

Directory State:
Binary: 0, Bucket ID: 1, Local Depth: 1, Values: [[16], [20], [12]]
Binary: 1, Bucket ID: 2, Local Depth: 1, Values: [[1], [13]]
Binary: 2, Bucket ID: 4, Local Depth: 1, Values: [[22], [26], [38]]
Binary: 3, Bucket ID: 5, Local Depth: 2, Values: [[3], [11], [19]]
Binary: 4, Bucket ID: 11, Local Depth: 1, Values: []
Binary: 5, Bucket ID: 12, Local Depth: 1, Values: []
Binary: 6, Bucket ID: 13, Local Depth: 1, Values: [[46]]
Binary: 7, Bucket ID: 14, Local Depth: 1, Values: [[47]]



In [33]:
#When expanding the directory, I can't seem to successfully store and clear all
# of the values in the buckets, as seen in insert 47, only the contents of bucket 4 were 
#cleared and stored

In [34]:
import numpy as np
import heapq
import os

#Generates data
def generate_data(num_pages, page_size, output_directory): 
    os.makedirs(output_directory, exist_ok=True) #Creates output directory
    for i in range(num_pages): #Creates multiple pages
        data = np.random.randint(0, 100, page_size) #Generates a page of random numbers between 0-100
        np.save(os.path.join(output_directory, f'page_{i}.npy'), data) #Saves the data into a numpy file
        print(f'Generated page {i}: {data}') #Print the generated page

#Sorts data
def sort_pages(input_directory, num_pages):
    for i in range(num_pages): #Iterates over each page
        data = np.load(os.path.join(input_directory, f'page_{i}.npy')) #Load data from the numpy files
        sorted_data = np.sort(data) #Sort the data
        np.save(os.path.join(input_directory, f'sorted_page_{i}.npy'), sorted_data) #Save the sorted data into a new file
        print(f'Sorted page {i}: {sorted_data}') #Prints sorted data

#Merges the sorted data        
def merge_pages(input_directory, num_pages, buffer_size, output_file):
    #Opens all the sorted data files
    open_files = [open(os.path.join(input_directory, f'sorted_page_{i}.npy'), 'rb') for i in range(num_pages)]
    with open(output_file, 'wb') as out_file: #Allows the program to edit the files
        buffers = [np.load(f) for f in open_files] #Load the sorted data into buffers
        min_heap = [] #Initialize min-heap
        for i, buffer in enumerate(buffers): #Iterate over each buffer with its index
            if buffer.size > 0: #Check if buffer is not empty
                heapq.heappush(min_heap, (buffer[0], i, 0)) #Push the first element of buffer into min-heap
        
        while min_heap: #While there are elements in min-heap
            val, buffer_index, element_index = heapq.heappop(min_heap) #Remove the smallest element from the heap
            out_file.write(val.tobytes()) #Write it to the output file
            next_element_index = element_index + 1 #Calculate the index of the next element in the same buffer
            if next_element_index < buffers[buffer_index].size: #Check if there are more elements in the buffer
                heapq.heappush(min_heap, (buffers[buffer_index][next_element_index], buffer_index, next_element_index))
                #Push the next value into min-heap
                
def external_merge_sort(input_directory, num_pages, page_size, buffer_size, output_file):
    sort_pages(input_directory, num_pages)
    merge_pages(input_directory, num_pages, buffer_size, output_file)

def display_data(file_path):
    data = np.fromfile(file_path, dtype=np.int32)
    print(f'Data in {file_path}: {data}')

#Example usage
output_directory = 'data'
generate_data(num_pages=8, page_size=2, output_directory=output_directory)
external_merge_sort(input_directory=output_directory, num_pages=8, page_size=2, buffer_size=10, output_file='sorted_data.bin')

# Display final merged data
display_data('sorted_data.bin')

Generated page 0: [20 92]
Generated page 1: [27 80]
Generated page 2: [63  3]
Generated page 3: [91 73]
Generated page 4: [69  2]
Generated page 5: [30 25]
Generated page 6: [56 11]
Generated page 7: [36 70]
Sorted page 0: [20 92]
Sorted page 1: [27 80]
Sorted page 2: [ 3 63]
Sorted page 3: [73 91]
Sorted page 4: [ 2 69]
Sorted page 5: [25 30]
Sorted page 6: [11 56]
Sorted page 7: [36 70]
Data in sorted_data.bin: [ 2  3 11 20 25 27 30 36 56 63 69 70 73 80 91 92]
